In [1]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
from peft import LoraConfig, get_peft_model
from huggingface_hub import snapshot_download

In [2]:
# # 스냅샷 로컬에 저장
# repo_id = "lemon-mint/gemma-ko-2b-instruct-v0.51"
# local_dir = "../model/gemma-ko-2b-instruct-v0.51"

# snapshot_download(repo_id=repo_id, local_dir=local_dir)

In [2]:
local_dir = "../model/kanana-nano-2.1b-instruct"

tokenizer = AutoTokenizer.from_pretrained(local_dir, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

In [3]:
df = pd.read_excel("./data/emotion_data.xlsx")
df = df.fillna('')

In [4]:
data_list = []
for _, row in df.iterrows():
    data_list.append({
        'emotion': row['감정_소분류'],
        '사람문장1': row['사람문장1'],
        '시스템문장1': row['시스템문장1'],
        '사람문장2': row['사람문장2'],
        '시스템문장2': row['시스템문장2'],
        '사람문장3': row['사람문장3'],
        '시스템문장3': row['시스템문장3'],
    })

dataset = Dataset.from_dict({key: [d[key] for d in data_list] for key in data_list[0].keys()})
dataset[0]

{'emotion': '노여워하는',
 '사람문장1': '일은 왜 해도 해도 끝이 없을까? 화가 난다.',
 '시스템문장1': '많이 힘드시겠어요. 주위에 의논할 상대가 있나요?',
 '사람문장2': '그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.',
 '시스템문장2': '혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요. ',
 '사람문장3': '',
 '시스템문장3': ''}

In [5]:
def formatting_prompts_func(examples):
    """
    멀티턴 대화를 다음 형식으로 변환:
    사람문장1
    시스템문장1
    사람문장2
    시스템문장2
    사람문장3
    시스템문장3
    """
    
    texts = []
    EOS_TOKEN = tokenizer.eos_token
    
    for i in range(len(examples['사람문장1'])):
        # 각 턴의 데이터 추출
        turns = []
        for turn in range(1, 4):  # 사람문장1~3, 시스템문장1~3
            person = examples[f'사람문장{turn}'][i].strip()
            system = examples[f'시스템문장{turn}'][i].strip()
            
            if person:  # 비어있지 않으면 추가
                turns.append(f"사람: {person}")
            if system:
                turns.append(f"시스템: {system}")
        
        # 대화를 하나의 텍스트로 연결
        if turns:
            text = "\n".join(turns) + EOS_TOKEN
            texts.append(text)
    
    return {"text": texts}

formatted_dataset = dataset.map(formatting_prompts_func, batched=True)
formatted_dataset[0]

Map:   0%|          | 0/51630 [00:00<?, ? examples/s]

{'emotion': '노여워하는',
 '사람문장1': '일은 왜 해도 해도 끝이 없을까? 화가 난다.',
 '시스템문장1': '많이 힘드시겠어요. 주위에 의논할 상대가 있나요?',
 '사람문장2': '그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.',
 '시스템문장2': '혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요. ',
 '사람문장3': '',
 '시스템문장3': '',
 'text': '사람: 일은 왜 해도 해도 끝이 없을까? 화가 난다.\n시스템: 많이 힘드시겠어요. 주위에 의논할 상대가 있나요?\n사람: 그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.\n시스템: 혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.<|eot_id|>'}

In [6]:
def tokenize_function(examples):
    tokens = tokenizer(examples["text"], padding=True, return_tensors="pt")
    return tokens

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_dataset[0]

Map:   0%|          | 0/51630 [00:00<?, ? examples/s]

{'emotion': '노여워하는',
 '사람문장1': '일은 왜 해도 해도 끝이 없을까? 화가 난다.',
 '시스템문장1': '많이 힘드시겠어요. 주위에 의논할 상대가 있나요?',
 '사람문장2': '그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.',
 '시스템문장2': '혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요. ',
 '사람문장3': '',
 '시스템문장3': '',
 'input_ids': [128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  128009,
  1280

In [7]:
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=[
        "q_proj", 
        "k_proj", 
        "v_proj"]
)
base_model = AutoModelForCausalLM.from_pretrained(local_dir, dtype=torch.bfloat16, device_map="auto", trust_remote_code=True)
model = get_peft_model(base_model, lora_config)

In [8]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_ds = split_dataset["train"]
eval_ds = split_dataset["test"]

In [9]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

args=TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 500,
        # num_train_epochs=1,
        learning_rate = 2e-4, 
        bf16 = True,
        seed = 1234,
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        output_dir = "outputs",

        logging_steps=100,
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False
)

trainer = Trainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
    args=args
)

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
100,1.917500,1.754203
200,1.682900,1.654666
300,1.661700,1.639871
400,1.640500,1.629212
